In [1]:
# from llama_index.core import  SimpleDirectoryReader, ServiceContext
# from llama_index.llms.huggingface import HuggingFaceLLM
# import torch

In [1]:
from dotenv import load_dotenv
import os
# Import necessary libraries for document processing, vector embeddings, and interaction with Pinecone
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from transformers import AutoTokenizer, AutoModel
# import torch
# import numpy as np
from langchain_community.vectorstores import Pinecone
# from langchain.prompts import PromptTemplate 
from pinecone import Pinecone, PodSpec
from langchain_core.documents.base import Document
# from langchain.chains.question_answering import load_qa_chain

In [2]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
pdf_directory='Policies/'
files = os.listdir(pdf_directory)

# Filter for PDF files
pdf_files = [file for file in files if file.endswith('.pdf')]

# Initialize an empty list to hold all pages from all PDFs
all_pages = []

# Load and split each PDF
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    pdf_loader = PyPDFLoader(pdf_path)
    pages = pdf_loader.load_and_split()
    all_pages.extend(pages)  # Add the pages from the current PDF to the list of all pages

# Now `all_pages` contains pages from all PDFs in the directory
print(f'Total PDFs loaded: {len(pdf_files)}')
print(f'Total pages loaded: {len(all_pages)}')
print(type(all_pages[0]))



Total PDFs loaded: 4
Total pages loaded: 43
<class 'langchain_core.documents.base.Document'>


In [4]:
# Combine page contents into a single context string for processing
context = "\n".join(str(p.page_content) for p in all_pages)

In [5]:
# Split the combined context into manageable chunks for embedding generation
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3200, chunk_overlap=400)
texts = text_splitter.split_text(context)

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np



def generate_embeddings(texts):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    """
    Generate embeddings for a list of texts using the SentenceTransformer model.
    
    Parameters:
    texts (list of str): A list of sentences for which to generate embeddings.
    
    Returns:
    np.ndarray: A NumPy array of shape (n_texts, embedding_size) containing the sentence embeddings.
    """
    # The encode method directly returns the embeddings as a NumPy array
    embeddings = model.encode(texts)
    return embeddings


In [7]:
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pinecone_client.Index("demo")

In [8]:
# Map document IDs to texts and upsert embeddings into Pinecone
id_to_text = {}  # Dictionary to map IDs to texts
for i, text in enumerate(texts):
    embedding_list = generate_embeddings([text])[0].tolist()
    document_id = str(i)
    pinecone_index.upsert(vectors=[(document_id, embedding_list)])
    id_to_text[document_id] = text

In [9]:
query_text="how many leaves can employee can avail ?"
query_embedding = generate_embeddings([query_text])[0].tolist()

In [19]:


query_results = pinecone_index.query(vector=query_embedding, top_k=3)

In [20]:
documents = [
    Document(
        page_content=id_to_text.get(match['id'], "Content not found"),
        metadata={"score": match['score'], "page": match['id']}
    )
    for match in query_results["matches"]
    if match['id'] in id_to_text  # Ensure the ID exists in id_to_text
]


In [21]:
# Combine page contents into a single context string for processing
context = "\n".join(str(p.page_content) for p in documents)

In [22]:
print(documents[0])

page_content='• Total  31 Leaves  Annually  \n \n \n21 PL \nKind  of Leave  \n5 SL \n 5 CL\n5  \n  \nCasual  Leaves - 5 in a Year  \n \n \nCL leave  may  be used  : \n• For personal  work/family  engagements  \n• CL is not carried  forward  for the next  Leave  Year \n• CL is not en-cashable  \n• CL can be taken  for a minimum  period  of half day to a maximum  of 1.5 days  in a month.\n6  \n  \n \nSick Leaves  – 5 in a year  \n \nSL leave  may  be used  and intimated :  \n• For medical  issues  \n• If any employee  avails  more  than  3 sick leaves,  in continuity  he/she  needs  to present  Medical  \ncertificate  by an Authorized  Doctor.  \n• SL is not carried  forward  for the next  Leave  Year \n• SL is not en -cashable\n7  \n  \nPrivileged  Leaves  – 21 in a year  \n \n• Employees  can earn  21 PL in a year  w.e.f.  Date  of Joining  \n• The entitlement  is accrued  at the rate of 1.75  days  per completed  month  of service.  \n• The approval  and scheduling  of such  time  off

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer


# Load your model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Update this to your actual model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



query="how many leaves can employee can avail ?"


context = " ".join([doc.page_content for doc in documents])

enhanced_query = f"You are Chatbot you have to give precise and accurate results .Given the context: {context}, respond with most accurate result and Most relevent result of : {query}"

    # Encode the enhanced query
input_ids = tokenizer.encode(enhanced_query, return_tensors="pt", truncation=True, max_length=200)

    # Generate a response
output_ids = model.generate(input_ids, max_length=210, num_return_sequences=1, temperature=1, do_sample=True)

    # Decode the output to text
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)


print("Response:", response)


Response: You are Chatbot you have to give precise and accurate results .Given the context: • Total  31 Leaves  Annually  
 
 
21 PL 
Kind  of Leave  
5 SL 
 5 CL
5  
  
Casual  Leaves - 5 in a Year  
 
 
CL leave  may  be used  : 
• For personal  work/family  engagements  
• CL is not carried  forward  for the next  Leave  Year 
• CL is not en-cashable  
• CL can be taken  for a minimum  period  of half day to a maximum  of 1.5 days  in a month.
6  
  
 
Sick Leaves  – 5 in a year  
 
SL leave  may  be used  and intimated :  
• To cover    medical   leave (medical emer


In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


context = " ".join([doc.page_content for doc in documents])  # Assuming 'documents' is defined

enhanced_query = f"You are Chatbot you have to give precise and accurate results .Given the context: {context}, respond with the most accurate result . Most relevant result of: {query_text}"

# Encode the enhanced query
input_ids = tokenizer.encode(context, return_tensors="pt", truncation=True, max_length=260)

# Generate a response with tuned hyperparameters
output_ids = model.generate(
    input_ids,
    max_length=270,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.1,  # Adjust for diversity vs confidence
    top_k=20,  # Top-k sampling
    top_p=0.95,  # Nucleus sampling
    repetition_penalty=1.2,  # Adjust repetition penalty
)

# Decode the output to text
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Response:", response)


Response: • Total  31 Leaves  Annually  
 
 
21 PL 
Kind  of Leave  
5 SL 
 5 CL
5  
  
Casual  Leaves - 5 in a Year  
 
 
CL leave  may  be used  : 
• For personal  work/family  engagements  
• CL is not carried  forward  for the next  Leave  Year 
• CL is not en-cashable  
• CL can be taken  for a minimum  period  of half day to a maximum  of 1.5 days  in a month.
6  
  
 
Sick Leaves  – 5 in a year  
 
SL leave  may  be used  and intimated :  
• For medical  issues  
• If any employee  avails  more  than  3 sick leaves,  in continuity  he/she  needs  to present  Medical  
certificate  by an Authorized  Doctor.  
• SL is not carried  forward  for the next  Leave  Year 
• SL is not en -cashable
7  
  
 
Total Sick
